In [2]:
import torch
import numpy as np
import multiprocessing as mp
import graph_alg

In [3]:
def sinkhorn_knopp(matrix, max_iter=1000, tol=1e-6):
    for _ in range(max_iter):
        matrix /= matrix.sum(axis=1, keepdims=True)  # 行ごとに正規化
        matrix /= matrix.sum(axis=0, keepdims=True)  # 列ごとに正規化
        
        if np.allclose(matrix.sum(axis=1), 1, atol=tol) and np.allclose(matrix.sum(axis=0), 1, atol=tol):
            break
    
    return matrix


def generate_permutation_array(N, num_goods):
    P = np.zeros((N, num_goods))
    for i in range(N): 
        P[i] = np.random.permutation(num_goods)
    
    return P


def sample_ranking(N, num_goods):
        """ 
        Samples ranked lists
        Arguments
            N: Number of samples
            prob: Probability of truncation       
        Returns:
            Ranked List of shape [N, Num_agents]
        """
        P = generate_permutation_array(N, num_goods) + 1
        
        return P / num_goods

    
def generate_batch(batch_size, num_goods):
        """
        Samples a batch of data from training
        Arguments
            batch_size: number of samples
            prob: probability of truncation
        Returns
            P: Agent's preferences, 
                P_{ij}: How much Agent-i prefers to be Good-j
        """
        N = batch_size * num_goods
        P = sample_ranking(N, num_goods).reshape(-1, num_goods, num_goods)
        return P

In [59]:
np.random.seed(0)
batch_size = 5
num_goods = 4
randomassignment = np.random.rand(batch_size, 4, 4)
RandomAssignment = np.array([sinkhorn_knopp(matrix) for matrix in randomassignment])
preferences = generate_batch(batch_size, num_goods)
P = torch.tensor(RandomAssignment, dtype=torch.float32)[:4, :, :]
Preferences = torch.tensor(preferences, dtype=torch.float32)[:4, :, :]

ev = graph_alg.compute_ev(P, Preferences)
violations, cpucount = ev.execute_all_cycles_batch()
print("P: ", P)
print("Preferences: ", Preferences)
print(violations)
print(cpucount)

Process SpawnPoolWorker-444:
Process SpawnPoolWorker-448:
Process SpawnPoolWorker-447:
Process SpawnPoolWorker-443:
Traceback (most recent call last):
Process SpawnPoolWorker-445:
  File "/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
    ~~~~~~~~^^
Process SpawnPoolWorker-442:
  File "/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ~~~~^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/pool.py

KeyboardInterrupt: 